In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
from keras.optimizers import SGD, Adam
from keras.layers import *
from keras.models import Sequential, Model
from sklearn.model_selection import *
from sklearn.metrics import *
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Flatten
from keras.applications import MobileNet

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
def load_images():
  path = "/content/drive/MyDrive/PCOSGen-train/PCOSGen-train/images"
  images = os.listdir(path)
  x = []
  for img in images:
    temp = cv2.imread(os.path.join(path ,img))
    twmp=cv2.resize(temp,(256,256))

    x.append(temp)

  return x

In [ ]:
df = pd.read_excel("/content/drive/MyDrive/PCOSGen-train/PCOSGen-train/class_label.xlsx")
df.head()

,imagePath,Healthy
0,100image13.jpg,1
1,100image2.jpg,1
2,100image65.jpg,0
3,100image71.jpg,0
4,100image83.jpg,0


In [ ]:
##ANN model

def ann():
  model=Sequential()
  model.add(Flatten(input_shape=(256,256,3)))
  model.add(Dense(128, activation='relu'))
  model.add(Dense(64,activation='relu'))
  model.add(Dense(1,activation='sigmoid'))

  return model


In [ ]:
def mobilenet(input_shape=(256,256,3)):
  base_model=MobileNet(weights='imagenet',include_top='False', input_shape=input_shape)
  model=Sequantial()
  model.add(base_model)
  model.add(Flatten())
  model.add(Dense(128, activation='relu'))
  model.add(Dense(64, activation='relu'))
  model.add(Dense(1, activation='sigmoid'))
  return model

In [ ]:
image_names = df['imagePath'].values
df_filtered = df[df['imagePath'].isin(image_names)]


In [ ]:
x=load_images()

In [ ]:
images = []
labels = []
for index, row in df_filtered.iterrows():
    image_name = row['imagePath']
    label = row['Healthy']
    image_index = np.where(np.array(image_names) == image_name)[0][0]
    image = x[image_index]
    resized_image = cv2.resize(image, (256, 256))
    images.append(resized_image)
    labels.append(label)

In [ ]:
images = np.array(images)
labels = np.array(labels)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)


In [ ]:
y_train_reshaped = np.expand_dims(np.expand_dims(y_train, axis=-1), axis=-1)
y_test_reshaped = np.expand_dims(np.expand_dims(y_test, axis=-1), axis=-1)

batch_size=16
epochs=10


In [ ]:
# print("Training ANN model...")

# # Define batch size and number of epochs
# batch_size = 32
# epochs = 10  # Adjust as needed

# ann_model=ann()
# ann_model.compile(optimizer=SGD(lr=0.01, momentum=0.9), loss='binary_crossentropy', metrics=['accuracy'])

# # Iterate over the dataset in batches
# for epoch in range(epochs):
#     print(f"Epoch {epoch+1}/{epochs}")
#     for batch_index in range(0, len(x_train), batch_size):
#         x_batch = x_train[batch_index:batch_index+batch_size]
#         y_batch = y_train[batch_index:batch_index+batch_size]

#         # Train on the current batch
#         ann_model.train_on_batch(x_batch, y_batch)

#     # Evaluate the model at the end of each epoch
#     loss, accuracy = ann_model.evaluate(x_test, y_test)
#     print(f"Validation Loss: {loss}, Validation Accuracy: {accuracy}")

# print("Training finished.")

Training ANN model...


Epoch 1/10



20/20 [==============================] - 9s 69ms/step - loss: 0.5924 - accuracy: 0.7391
Validation Loss: 0.5923860669136047, Validation Accuracy: 0.739062488079071
Epoch 2/10
20/20 [==============================] - 3s 113ms/step - loss: 0.5768 - accuracy: 0.7391
Validation Loss: 0.5767970085144043, Validation Accuracy: 0.739062488079071
Epoch 3/10
20/20 [==============================] - 2s 89ms/step - loss: 0.5756 - accuracy: 0.7391
Validation Loss: 0.5756357312202454, Validation Accuracy: 0.739062488079071
Epoch 4/10


KeyboardInterrupt: 

In [ ]:
from sklearn.utils import class_weight
from keras.callbacks import EarlyStopping
from keras.preprocessing.image import ImageDataGenerator

class_labels=np.unique(y_train)
class_weights = class_weight.compute_class_weight(class_weight='balanced',classes=class_labels, y=y_train)
class_weights=dict(zip(np.unique(y_train), class_weights)),class_weights

In [ ]:
datagen = ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    rescale=1.0 / 255  # Rescale pixel values to [0, 1]
)

In [ ]:
def custommodel():
    model = Sequential()
    model.add(Conv2D(8, (3, 3), activation='relu', input_shape=(256, 256, 3)))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(16, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(32, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))

    return model

In [ ]:
model=custommodel()
model.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.0001), metrics=['accuracy'])

In [ ]:
early_stopping = EarlyStopping(monitor='val_loss', patience=10)


In [ ]:
steps_per_epoch = len(x_train) // batch_size
batch_size = 16  # Reduce batch size to conserve memory
epochs = 10

In [ ]:
history = model.fit_generator(
    generator=datagen.flow(x_train[:500], y_train[:500], batch_size=batch_size),
    steps_per_epoch=steps_per_epoch,
    epochs=epochs,
    validation_data=(x_test, y_test),
    callbacks=[early_stopping],
)

C:\Users\hp\AppData\Local\Temp\ipykernel_15060\2761170086.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(


Epoch 1/10
32/80 [===========>..................] - ETA: 27s - loss: 0.5933 - accuracy: 0.7320WARNING:tensorflow:Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches (in this case, 800 batches). You may need to use the repeat() function when building your dataset.


80/80 [==============================] - 32s 372ms/step - loss: 0.5933 - accuracy: 0.7320 - val_loss: 52.8794 - val_accuracy: 0.7391


In [ ]:
from keras.applications.densenet import DenseNet121
from keras.applications import VGG16, ResNet50



In [ ]:
# Convert labels to categorical
from keras.utils import to_categorical
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Input, BatchNormalization, Activation, Concatenate, AveragePooling2D
from keras.applications.densenet import DenseNet121

y_traincatg = to_categorical(y_train, num_classes=2)
y_testcatg = to_categorical(y_test, num_classes=2)

In [ ]:
def dense():
    base_model = DenseNet121(include_top=False, weights='imagenet', input_shape=(256, 256, 3))

    for layer in base_model.layers:
        layer.trainable = False

    x = base_model.output
    x = Flatten()(x)
    x = Dense(1024, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)
    # x = Dense(512, activation='relu')(x)
    # x = BatchNormalization()(x)
    # x = Dropout(0.5)(x)




    predictions = Dense(1, activation='sigmoid')(x)

    model = Model(inputs=base_model.input, outputs=predictions)

    return model

In [ ]:
def vggmodel():
    base_model = VGG16(include_top=False, weights='imagenet', input_shape=(256, 256, 3))

    for layer in base_model.layers:
        layer.trainable = False

    x = base_model.output
    x = Flatten()(x)
    x = Dense(1024, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)

    predictions = Dense(1, activation='sigmoid')(x)

    model = Model(inputs=base_model.input, outputs=predictions)

    return model

In [ ]:
def cnn_model():
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(256, 256, 3)))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))
    return model


In [ ]:
def resnet():
    base_model = ResNet50(include_top=False, weights='imagenet', input_shape=(256, 256, 3))

    for layer in base_model.layers:
        layer.trainable = False

    x = base_model.output
    x = Flatten()(x)
    x = Dense(1024, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)

    predictions = Dense(1, activation='sigmoid')(x)

    model = Model(inputs=base_model.input, outputs=predictions)

    return model

In [ ]:
from keras.models import Model
from keras.layers import concatenate

In [ ]:
#Compilinhg seperately and fitting because the next few ensembled model is fully flawed smh

densenet_model = dense()
cnn = cnn_model()
vgg = vggmodel()

# Compile the individual models
densenet_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
cnn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
vgg.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

58889256/58889256 [==============================] - 3s 0us/step


In [ ]:
densenet_model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=20, batch_size=64)

Epoch 1/20
40/40 [==============================] - 14s 341ms/step - loss: 0.4447 - accuracy: 0.7941 - val_loss: 0.6985 - val_accuracy: 0.7547
Epoch 2/20
40/40 [==============================] - 13s 338ms/step - loss: 0.4450 - accuracy: 0.7922 - val_loss: 0.6924 - val_accuracy: 0.7688
Epoch 3/20
40/40 [==============================] - 13s 327ms/step - loss: 0.4341 - accuracy: 0.8039 - val_loss: 0.6514 - val_accuracy: 0.7453
Epoch 4/20
40/40 [==============================] - 13s 317ms/step - loss: 0.3960 - accuracy: 0.8246 - val_loss: 0.6491 - val_accuracy: 0.7391
Epoch 5/20
40/40 [==============================] - 12s 314ms/step - loss: 0.3991 - accuracy: 0.8184 - val_loss: 0.7421 - val_accuracy: 0.7547
Epoch 6/20
40/40 [==============================] - 13s 316ms/step - loss: 0.4036 - accuracy: 0.8156 - val_loss: 0.7443 - val_accuracy: 0.7641
Epoch 7/20
40/40 [==============================] - 13s 322ms/step - loss: 0.3763 - accuracy: 0.8316 - val_loss: 0.6975 - val_accuracy: 0.7609

In [ ]:
cnn.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=10, batch_size=64)


Epoch 1/10
40/40 [==============================] - 14s 152ms/step - loss: 108.8670 - accuracy: 0.6594 - val_loss: 0.6112 - val_accuracy: 0.7391
Epoch 2/10
40/40 [==============================] - 6s 148ms/step - loss: 0.6054 - accuracy: 0.7297 - val_loss: 0.5788 - val_accuracy: 0.7516
Epoch 3/10
40/40 [==============================] - 6s 144ms/step - loss: 0.5571 - accuracy: 0.7371 - val_loss: 0.5827 - val_accuracy: 0.7625
Epoch 4/10
40/40 [==============================] - 5s 137ms/step - loss: 0.5108 - accuracy: 0.7547 - val_loss: 0.5881 - val_accuracy: 0.7531
Epoch 5/10
40/40 [==============================] - 6s 144ms/step - loss: 0.4715 - accuracy: 0.7746 - val_loss: 0.5998 - val_accuracy: 0.7391
Epoch 6/10
40/40 [==============================] - 5s 133ms/step - loss: 0.4292 - accuracy: 0.7863 - val_loss: 0.6533 - val_accuracy: 0.7375
Epoch 7/10
40/40 [==============================] - 5s 135ms/step - loss: 0.3946 - accuracy: 0.8145 - val_loss: 0.6591 - val_accuracy: 0.7109
Epo

In [ ]:


def create_ensemble_model():
    # Get individual models
    densenet_model = dense()
    cnn = cnn_model()
    vgg=vggmodel()

    # Create an input layer for each model
    input_densenet = densenet_model.input
    input_cnn = cnn.input
    input_vgg=vgg.input
    # Get the output of each model
    output_densenet = densenet_model.output
    output_cnn = cnn.output
    output_vgg=vgg.output

    # Concatenate the outputs of the models
    merged_output = concatenate([output_densenet, output_cnn, output_vgg])

    # Add a fully connected layer for the ensemble
    merged_output = Dense(64, activation='relu')(merged_output)
    merged_output = Dropout(0.5)(merged_output)
    predictions = Dense(1, activation='sigmoid')(merged_output)

    # Create the ensemble model
    ensemble_model = Model(inputs=[input_densenet, input_cnn, input_vgg], outputs=predictions)

    return ensemble_model


In [ ]:
ensemble_model = create_ensemble_model()

# Compile the ensemble model
ensemble_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

ensemble_model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=10, batch_size=64)

58889256/58889256 [==============================] - 1s 0us/step
Epoch 1/10


ValueError: in user code:

    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1401, in train_function  *
        return step_function(self, iterator)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1384, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1373, in run_step  **
        outputs = model.train_step(data)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1150, in train_step
        y_pred = self(x, training=True)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/input_spec.py", line 219, in assert_input_compatibility
        raise ValueError(

    ValueError: La

In [ ]:
batch_size = 16
epochs = 10
model=dense()
Momentum = SGD(lr=0.01, momentum=0.9)
model.compile(optimizer=Momentum, loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:

model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=10, batch_size=64)


Epoch 1/10
40/40 [==============================] - 21s 346ms/step - loss: 0.9075 - accuracy: 0.6047 - val_loss: 1.3821 - val_accuracy: 0.4172
Epoch 2/10
40/40 [==============================] - 12s 305ms/step - loss: 0.6666 - accuracy: 0.6809 - val_loss: 0.7262 - val_accuracy: 0.7125
Epoch 3/10
40/40 [==============================] - 12s 292ms/step - loss: 0.6985 - accuracy: 0.6816 - val_loss: 1.0122 - val_accuracy: 0.7484
Epoch 4/10
40/40 [==============================] - 12s 295ms/step - loss: 0.7157 - accuracy: 0.6773 - val_loss: 0.7985 - val_accuracy: 0.6859
Epoch 5/10
40/40 [==============================] - 11s 278ms/step - loss: 0.6798 - accuracy: 0.6914 - val_loss: 0.6748 - val_accuracy: 0.7391
Epoch 6/10
40/40 [==============================] - 12s 289ms/step - loss: 0.6809 - accuracy: 0.6930 - val_loss: 0.7305 - val_accuracy: 0.5969
Epoch 7/10
40/40 [==============================] - 12s 290ms/step - loss: 0.6484 - accuracy: 0.7016 - val_loss: 0.6547 - val_accuracy: 0.7422

In [ ]:
def densecateg():
    base_model = DenseNet121(include_top=False, weights=None, input_shape=(256, 256, 3))

    x = base_model.output
    x = Flatten()(x)
    predictions = Dense(2, activation='softmax')(x)

    model = Model(inputs=base_model.input, outputs=predictions)
    return model

In [ ]:
batch_size = 64
epochs = 10
model=densecateg()
Momentum = SGD(lr=0.01, momentum=0.9)
model.compile(optimizer=Momentum, loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
# for epoch in range(epochs):
#     print(f"Epoch {epoch+1}/{epochs}")
#     for batch_index in range(0, len(x_train), batch_size):
#         x_batch = x_train[batch_index:batch_index+batch_size]
#         y_batch = y_traincatg[batch_index:batch_index+batch_size]

#         # Train on the current batch
#         model.train_on_batch(x_batch, y_batch)

#     # Evaluate the model at the end of each epoch
#     loss, accuracy = model.evaluate(x_test, y_testcatg)
#     print(f"Validation Loss: {loss}, Validation Accuracy: {accuracy}")

# print("Training finished.")

model.fit(x_train[:500], y_traincatg[:500], validation_data=(x_test, y_testcatg), epochs=10, batch_size=16)


Epoch 1/10
32/32 [==============================] - 48s 558ms/step - loss: 5.9578 - accuracy: 0.6760 - val_loss: 119523.8281 - val_accuracy: 0.7297
Epoch 2/10
32/32 [==============================] - 9s 292ms/step - loss: 6.1529 - accuracy: 0.6200 - val_loss: 137043.6250 - val_accuracy: 0.7063
Epoch 3/10
32/32 [==============================] - 10s 303ms/step - loss: 3.9172 - accuracy: 0.6820 - val_loss: 116191.5156 - val_accuracy: 0.5703
Epoch 4/10
32/32 [==============================] - 9s 299ms/step - loss: 2.9219 - accuracy: 0.6880 - val_loss: 4019.2351 - val_accuracy: 0.7312
Epoch 5/10
32/32 [==============================] - 9s 294ms/step - loss: 2.2821 - accuracy: 0.7020 - val_loss: 1797.9675 - val_accuracy: 0.6828
Epoch 6/10
32/32 [==============================] - 10s 302ms/step - loss: 2.4982 - accuracy: 0.6900 - val_loss: 195.6660 - val_accuracy: 0.6266
Epoch 7/10
32/32 [==============================] - 10s 310ms/step - loss: 1.1787 - accuracy: 0.7220 - val_loss: 55.9102 -